# ACI COLORIZATION NOTEBOOK

## S. Sharma 
## March 6, 2023 (based off initial script from 11/2021)
### For Python version 3.8.12 

This notebook is for "colorizing" autofocus context imager (ACI) images from SHERLOC. The colorization process essentially finds and matches keypoints between the ACI (grayscale) image and the WATSON (color) image of the same target and blends the colors, providing a kind of "merge" product. This is useful because ACI images are co-boresighted with the SHERLOC spectrometer and are the highest resolution, thus providing both spectral mapping and textural information. By contrast, the WATSON images are color images that are taken from different standoff distances and under various illumination conditions, thus providing information about the color of rock targets useful for mineral/rock type identification. The colorized ACI is a composite product that provides both color and texture information, and SHERLOC spectral maps can be overlaid on it for spectral-textural correlation. 

Notes:

* This notebook can be used with PIXL MCC images as well, which are similarly grayscale and provide XRF spatial mapping information. 
* This process works best abraded surface images, because more keypoints are found. It does not work well with natural surfaces. 
* OpenCV uses BGR space; matplotlib uses RGB so a color conversion is required to display images here. 

The steps of the notebook as are follows:

1. SETUP: Import all packages.
2. LABEL AND SHOW IMAGES: Each image is imported and displayed. The user should provide full paths to each.  
3. CONVERT IMAGES TO GRAY: Convert all images to grayscale.
4. DETECTOR: Create keypoints on each image using the BRISK keypoint detector and descriptor extractor (https://docs.opencv.org/3.4/de/dbf/classcv_1_1BRISK.html), and draw them and display the annotated images. 
5. FIND MATCHES: Using a FLANN-based matcher, find the number of matches between the ACI and each WATSON candidate. Draw these between images. 
6. APPLY: Apply the matches to find homography between images, warp, and generate the colorized ACI. Save product in the same folder that the notebook is in. 

References and Links:

1. Stefan Leutenegger, Margarita Chli, and Roland Yves Siegwart. Brisk: Binary robust invariant scalable keypoints. In Computer Vision (ICCV), 2011 IEEE International Conference on, pages 2548–2555. IEEE, 2011.
2. https://docs.opencv.org/3.4/ 
3. David G. Lowe. Distinctive Image Features from Scale-Invariant Keypoints. International Journal of Computer Vision, 2004. 
4. Vino Mahendran. Feature detection and matching with OpenCV. Francium Tech Blog, 2020. https://blog.francium.tech/feature-detection-and-matching-with-opencv-5fd2394a590. 


### 1. SETUP

Here, the Jupyter notebook is initialized and each needed library/module is imported. Absolute imports are used.

In [ ]:
# Jupyter
%load_ext autoreload
%autoreload 2
%matplotlib widget

# Setup
import os
os.chdir('..')

# Imports
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from os import listdir
import pathlib
from pathlib import Path

import argparse

import cv2
import numpy as np

print("All packages imported")


### 2. SET PATHS AND SHOW IMAGES

ACI and WATSONs of choice are imported and displayed. Insert the full path of the ACI as the src_img_path variable (replace current text with the correct path), and the full path of the WATSON as dest_img_path. 

In [ ]:
aci_path = '/Users/susharma/Documents/M2020/Projects/SIDNEY/DATA/SRLC/Bellegarde/ACI/SC3_0186_0683476704_113ECM_N0070000SRLC10500_0000LMJ01.PNG'
watson_path = '/Users/susharma/Documents/M2020/Projects/SIDNEY/DATA/SRLC/Bellegarde/WATSON/ASRLC_SOL0185NIAL_0683368184_652_0070000_SRLC00720__000ECM_J02.png'
aci = cv2.imread(aci_path)
watson = cv2.imread(watson_path)

plt.imshow(cv2.cvtColor(aci, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Original ACI')
plt.show()



In [ ]:
plt.close()
plt.imshow(cv2.cvtColor(watson, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Original WATSON')
plt.show()

### 3. CONVERT IMAGES TO GRAYSCALE

Convert to grayscale space. The minimum number of matches that is accepted can be changed here. 

In [ ]:
watson_color = watson.copy()
watson = cv2.cvtColor(watson, cv2.COLOR_BGR2GRAY)
aci = cv2.cvtColor(aci, cv2.COLOR_BGR2GRAY)

min_matches = 37

### 4. DETECTOR

Create keypoints on each image using the BRISK keypoint detector and descriptor extractor (https://docs.opencv.org/3.4/de/dbf/classcv_1_1BRISK.html), and draw them and display the annotated images.

In [ ]:
detector = cv2.BRISK_create()
kp1, des1 = detector.detectAndCompute(watson, None)
kp2, des2 = detector.detectAndCompute(aci, None)
kp1_img = cv2.drawKeypoints(watson, kp1, None, color=(255,255,0))
kp2_img = cv2.drawKeypoints(aci, kp2, None, color=(255,255,0))

plt.close()
plt.imshow(kp1_img)
plt.axis('off')
plt.title('Keypoints on WATSON')
plt.show()

In [ ]:
plt.close()
plt.imshow(kp2_img)
plt.axis('off')
plt.title('Keypoints on ACI')
plt.show()

### 5. FIND MATCHES

Using a FLANN-based matcher, find the number of matches between the ACI and each WATSON candidate. Draw these between images. The distance value can be changed here. 

In [ ]:
index_params = dict(algorithm=6,
                    table_number=6,
                    key_size=12,
                    multi_probe_level=2)
search_params = {}
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

img_matches = cv2.drawMatchesKnn(watson,kp1,aci,kp2,matches[:10],None,flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
plt.close()
plt.imshow(img_matches)
plt.axis('off')
plt.title('Matches')

plt.show()

In [ ]:
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

print('Number of Matches:')
print(len(good_matches))

### 6. APPLY 

Apply the matches to find homography between images, warp, and generate the colorized ACI. Save product. 

In [ ]:
if len(good_matches) > min_matches:
    print('I can colorize this, probably')
    src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    m, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
    colorized_aci = cv2.warpPerspective(watson_color, m, (aci.shape[1], aci.shape[0]))
    colorized_aci = cv2.cvtColor(colorized_aci, cv2.COLOR_BGR2HSV)
    colorized_aci[:,:,2] = aci.astype(np.uint8)
    colorized_aci = cv2.cvtColor(colorized_aci, cv2.COLOR_HSV2BGR)
    plt.close()
    plt.imshow(cv2.cvtColor(colorized_aci, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
else:
    print('I cannot colorize, here is your original image')
    plt.close()
    plt.imshow(cv2.cvtColor(aci, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
cv2.imwrite('colorized_aci.png', colorized_aci)